In [283]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import ensemble
from sklearn import feature_selection
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score

import xgboost

from sklearn.decomposition import PCA

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [284]:
train = pd.read_csv('train.csv')
train.drop('Id', axis = 1, inplace = True)

X = train.drop('SalePrice', axis = 1)
y = train['SalePrice']


In [285]:
def prepare_X(X, y):
    #########################去掉Outlier###############################
    
    outliers = [1298, 495, 249, 313, 335, 706, 185]
    X.drop(outliers, inplace = True)
    y = y.drop(outliers)
    
    #########################选择合适的features#########################
    
    cols = [
             'OverallQual','ExterQual','CentralAir','1stFlrSF','GrLivArea','BsmtFullBath','BsmtHalfBath',
             'KitchenQual','TotRmsAbvGrd','Fireplaces','PavedDrive','TotalBsmtSF','GarageArea','YearRemodAdd',
             'YearBuilt','BsmtFinSF1','2ndFlrSF','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr', 'LotArea',
             'PoolArea','MSZoning','Neighborhood','Exterior1st','Exterior2nd','Foundation','SaleType', 'WoodDeckSF',
             'OpenPorchSF','LotFrontage','MSSubClass', 'LotShape','LandContour','LandSlope','Condition1','BldgType',
             'OverallCond','RoofStyle', 'ExterCond', 'BsmtQual','BsmtCond', 'BsmtFinSF2', 'BsmtUnfSF','HeatingQC',
             'Electrical', 'LowQualFinSF', 'Functional','GarageType', 'GarageFinish','GarageCond'
    ]
    X_prepared = X[cols] 
    
    #########################填补Missingness###############################
    X_prepared['BsmtFinSF2'].fillna(0, inplace = True)
    
    num_col = ['LotFrontage', 'MasVnrArea']
    
    for num in num_col:
        X_prepared[num] = X.groupby(['Neighborhood'])[num].apply(lambda x: x.fillna(x.median()))
    
    cat_col = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
    
    for cat in cat_col:
        X_prepared[cat] = X.groupby(['Neighborhood'])[cat].apply(lambda x:x.fillna(x.value_counts().index[0]))
    
    X_prepared['BsmtQual'] = X_prepared['BsmtQual'].fillna('No_Basement')
    X_prepared['BsmtCond'] = X_prepared['BsmtCond'].fillna('No_Basement')
    
    #########################处理一下Column############################
    
    ########Categorical######
    
    Qual_dict = {'Ex':3, 'Gd': 2, 'TA': 1, 'Fa': 0}
    Qual_dict2 = {'Ex':4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po':0}
    Qual_dict3 = {'Ex':4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'No_Basement':0}
    Qual_dict4 = {'Gd':4, 'TA': 3, 'Fa': 2, 'Po': 1, 'No_Basement':0}
    Qual_dict5 = {'Fin':2, 'RFn':1, 'Unf':0}
    
    # GarageCond
    X_prepared['GarageCond'] = X_prepared['GarageCond'].replace(Qual_dict2)
    # GarageQual
    X_prepared['GarageQual'] = X_prepared['GarageQual'].replace(Qual_dict2)
    # GarageFinish
    X_prepared['GarageFinish'] = X_prepared['GarageFinish'].replace(Qual_dict5)
    # HeatingQC
    X_prepared['HeatingQC'] = X_prepared['HeatingQC'].replace(Qual_dict2)
    # BsmtCond
    X_prepared['BsmtCond'] = X_prepared['BsmtCond'].replace(Qual_dict4)
    # BsmtQual
    X_prepared['BsmtQual'] = X_prepared['BsmtQual'].replace(Qual_dict3)
    # ExterCond
    X_prepared['ExterCond'] = X_prepared['ExterCond'].replace(Qual_dict2)
    # ExterQual
    X_prepared['ExterQual'] = X_prepared['ExterQual'].replace(Qual_dict)
    # KitchenQual
    X_prepared['KitchenQual'] = X_prepared['KitchenQual'].replace(Qual_dict)
    # LandSlope
    X_prepared['LandSlope'] = X_prepared['LandSlope'].replace({'Sev':2, 'Mod':1, 'Gtl':0})
    # ONE HOT
    onehot_col = ['MSZoning', 'Neighborhood','Exterior1st','Exterior2nd','Foundation','CentralAir',
                'PavedDrive', 'SaleType','MSSubClass', 'LotShape', 'LandContour','Condition1','BldgType',
                'RoofStyle','Electrical', 'Functional','GarageType'
                    ]
    X_prepared = pd.get_dummies(X_prepared, columns = onehot_col)
    
    ########Numerical########
    
    high_skew_col = ['LotArea', 'WoodDeckSF', '1stFlrSF', 'TotalBsmtSF', 'OpenPorchSF', 'LotFrontage', 
                     'MasVnrArea', 'BsmtFinSF2']
    for col in high_skew_col:
        X_prepared[col] = np.log1p(X_prepared[col])
        
    #########################新feature################################
    
    X['has_2ndsf'] = (X['2ndFlrSF'] != 0).astype(np.int)
    X_prepared['Bath'] = X_prepared['FullBath'] + 0.5 * X_prepared['HalfBath']
    X_prepared['has_pool'] = (X_prepared['PoolArea'] == 0).astype(np.int)
    X_prepared['has_remodel'] = (X_prepared['YearRemodAdd'] != X_prepared['YearBuilt']).astype(np.int)
    X_prepared['BsmtBath'] = X_prepared['BsmtFullBath'] + 0.5 * X_prepared['BsmtHalfBath']
    X_prepared['Overall'] = X_prepared['OverallCond'] + X_prepared['OverallQual']
    X_prepared['has_MasVnr'] = (X_prepared['MasVnrArea'] == 0).astype(np.int)
    X_prepared['Exter'] = X_prepared['ExterCond'] + X_prepared['ExterQual']
    X_prepared['Bsmt'] = X_prepared['BsmtCond'] + X_prepared['BsmtQual']
    X_prepared['has_BsmtFinSF2'] = (X_prepared['BsmtFinSF2'] != 0).astype(np.int)
    X_prepared['has_lowqualitysf'] = (X_prepared['LowQualFinSF'] != 0).astype(np.int)
    X_prepared['Garage'] = X_prepared['GarageQual'] + X_prepared['GarageCond']
    
    #########################不要的column扔了############################
    to_drop = ['YearBuilt', 'PoolArea', 'LowQualFinSF']
    X_prepared.drop(to_drop, axis = 1, inplace = True)
    return X_prepared, y


In [286]:
X_prepared, y_prepared = prepare_X(X, y)
X_prepared.head()


,OverallQual,ExterQual,1stFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,KitchenQual,TotRmsAbvGrd,Fireplaces,TotalBsmtSF,...,has_pool,has_remodel,BsmtBath,Overall,has_MasVnr,Exter,Bsmt,has_BsmtFinSF2,has_lowqualitysf,Garage
0,7,2,6.753438,1710,1,0,2,8,0,6.753438,...,1,0,1.0,12,0,4,6,0,0,4
1,6,1,7.141245,1262,0,1,1,6,1,7.141245,...,1,0,0.5,14,1,3,6,0,0,4
2,7,2,6.825460,1786,1,0,2,6,1,6.825460,...,1,1,1.0,12,0,4,6,0,0,4
3,7,1,6.869014,1717,1,0,2,7,1,6.629363,...,1,1,1.0,12,1,3,6,0,0,4
4,8,2,7.044033,2198,1,0,2,9,1,7.044033,...,1,0,1.0,13,0,4,6,0,0,4


In [287]:
# Feature Selection
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def neg_rmse(estimator, X, y):
    y_true = y
    y_pred = estimator.predict(X)
    return (-1) * np.sqrt(np.mean((y_true - y_pred) ** 2))

negative_rmse = make_scorer(neg_rmse)

estimator = ensemble.GradientBoostingRegressor()
selector = feature_selection.RFECV(estimator, cv = 5, n_jobs = -1, scoring = neg_rmse)
selector = selector.fit(X_prepared, y_prepared)

print("The number of selected features is: {}".format(selector.n_features_))

features_keep = X_prepared.columns.values[selector.support_]

X_prepared = selector.transform(X_prepared)

The number of selected features is: 113


In [288]:
# 随便找一个试试
gb_reg = ensemble.GradientBoostingRegressor()
gb_reg.fit(X_prepared, y_prepared)

GradientBoostingRegressor()

In [289]:
cv_score = (cross_val_score(gb_reg, X_prepared, y_prepared, cv = 5)).mean()
mse = mean_squared_error(gb_reg.predict(X_prepared), y_prepared)

print('CV score is:', cv_score)
print('MSE is:', mse)

CV score is: 0.8762622308148931
MSE is: 226218733.41989654


In [290]:
# Check feature importances
feature_importance = pd.DataFrame({'Name': features_keep, 'Importance': gb_reg.feature_importances_}).sort_values('Importance'
                                                                                    ,ascending = False)
feature_importance.head()

,Name,Importance
0,OverallQual,0.493910
3,GrLivArea,0.115254
9,TotalBsmtSF,0.072411
1,ExterQual,0.044482
12,BsmtFinSF1,0.034447


# Model Selection

In [208]:
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn.kernel_ridge import KernelRidge

def neg_rmse(y_true, y_pred):
    return (-1) * np.sqrt(np.mean((y_true - y_pred) ** 2))


In [209]:
MLA = [
    #Linear Model
    linear_model.Lasso(),
    linear_model.Ridge(),
    linear_model.ElasticNet(),

    #Kernel Ridge
    KernelRidge(),
    
    #Tree
    tree.DecisionTreeRegressor(),
    
    #Ensemble
    ensemble.RandomForestRegressor(),
    ensemble.GradientBoostingRegressor(),
    
    #XGB
    xgboost.XGBRegressor()
]
scorer = make_scorer(neg_rmse)
df_columns = ['Name', 'Parameters', 'CV score mean']
df = pd.DataFrame(columns = df_columns)
from sklearn.model_selection import cross_validate
for i in np.arange(len(MLA)):
    model = MLA[i]
    model_name = model.__class__.__name__
    model_parameters = str(model.get_params())
    cv_results = (cross_val_score(model, X_prepared, y_prepared, cv = 5, scoring=scorer)).mean()
    df.loc[i,:] = [model_name, model_parameters, cv_results]
    

In [210]:
df.sort_values('CV score mean', ascending = False)


,Name,Parameters,CV score mean
6,GradientBoostingRegressor,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ...",-27059.5
7,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",-28203.5
5,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",-28580.6
3,KernelRidge,"{'alpha': 1, 'coef0': 1, 'degree': 3, 'gamma':...",-29652.9
1,Ridge,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",-29782.6
0,Lasso,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",-29973.1
2,ElasticNet,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",-31841.1
4,DecisionTreeRegressor,"{'ccp_alpha': 0.0, 'criterion': 'mse', 'max_de...",-38537.1


# Search for hyperparameters

## XGB

In [228]:
#Search for parameters
xgb_reg = xgboost.XGBRegressor(
    eta = 0.2,
    max_depth = 4
)
parameters = {
    'subsample':[0.5, 0.8, 1]
}
cv = GridSearchCV(estimator=xgb_reg, param_grid=parameters, scoring='r2', n_jobs = -1, cv = 5)

cv.fit(X_prepared, y_prepared)

cv.best_params_

{'subsample': 1}

In [239]:
#Search for parameters
gb_reg = ensemble.GradientBoostingRegressor(
    n_estimators=800,
    learning_rate=0.05,
    subsample=0.6,
    criterion = 'mse'
)
parameters = {
    'criterion':['friedman_mse', 'mse', 'mae']
}
cv = GridSearchCV(estimator=gb_reg, param_grid=parameters, scoring='r2', n_jobs = -1, cv = 5)

cv.fit(X_prepared, y_prepared)

cv.best_params_

{'criterion': 'mse'}

In [253]:
#Search for parameters
rf_reg = ensemble.RandomForestRegressor(
    n_estimators=400,
    ccp_alpha=0.01,
    max_features='log2'
)
parameters = {
    'max_features':['auto', 'sqrt', 'log2']
}
cv = GridSearchCV(estimator=gb_reg, param_grid=parameters, scoring='r2', n_jobs = -1, cv = 5)

cv.fit(X_prepared, y_prepared)

cv.best_params_

{'max_features': 'log2'}

# Train the model

In [291]:
xgb_reg = xgboost.XGBRegressor(
    eta = 0.2,
    max_depth = 4
)
gb_reg = ensemble.GradientBoostingRegressor(
    n_estimators=800,
    learning_rate=0.05,
    subsample=0.6,
    criterion = 'mse'
)
rf_reg = ensemble.RandomForestRegressor(
    n_estimators=400,
    ccp_alpha=0.01,
    max_features='log2'
)

models = [xgb_reg, gb_reg, rf_reg]
for i in models:
    i.fit(X_prepared, y_prepared)

# Prediction

In [292]:
test = pd.read_csv('test.csv')


In [293]:
def prepare_test(X):
    #########################选择合适的features#########################
    
    cols = [
             'OverallQual','ExterQual','CentralAir','1stFlrSF','GrLivArea','BsmtFullBath','BsmtHalfBath',
             'KitchenQual','TotRmsAbvGrd','Fireplaces','PavedDrive','TotalBsmtSF','GarageArea','YearRemodAdd',
             'YearBuilt','BsmtFinSF1','2ndFlrSF','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr', 'LotArea',
             'PoolArea','MSZoning','Neighborhood','Exterior1st','Exterior2nd','Foundation','SaleType', 'WoodDeckSF',
             'OpenPorchSF','LotFrontage','MSSubClass', 'LotShape','LandContour','LandSlope','Condition1','BldgType',
             'OverallCond','RoofStyle', 'ExterCond', 'BsmtQual','BsmtCond', 'BsmtFinSF2', 'BsmtUnfSF','HeatingQC',
             'Electrical', 'LowQualFinSF', 'Functional','GarageType', 'GarageFinish','GarageCond','GarageQual',
             'MasVnrArea'
    ]
    X_prepared = X[cols] 
    
    #########################处理一下Missingness############################
    
    X_prepared['BsmtQual'] = X_prepared['BsmtQual'].fillna('No_Basement')
    X_prepared['BsmtCond'] = X_prepared['BsmtCond'].fillna('No_Basement')
    
    # Categorical的用mode， 根据 Neighborhood
    # Numerical的用median， 根据 Neighborhood
    X_prepared['GarageFinish'] = X.groupby(['Neighborhood'])['GarageFinish'].apply(lambda x:x.fillna(x.value_counts().index[0]))
    X_prepared['GarageCond'] = X.groupby(['Neighborhood'])['GarageCond'].apply(lambda x:x.fillna(x.value_counts().index[0]))
    X_prepared['GarageQual'] = X.groupby(['Neighborhood'])['GarageQual'].apply(lambda x:x.fillna(x.value_counts().index[0]))
    
    mode_col = ['BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'TotalBsmtSF']
    median_col = ['GarageArea', 'BsmtFinSF1', 'LotFrontage', 'MasVnrArea', 'BsmtFinSF2', 'BsmtUnfSF' 
                     ]
                     #'GarageFinish', 'MasVnrArea']

    for i in mode_col:
        X_prepared[i] = X.groupby(['Neighborhood'])[i].apply(lambda x:x.fillna(x.value_counts().index[0]))
    for k in median_col:
        X_prepared[k] = X.groupby(['Neighborhood'])[i].apply(lambda x: x.fillna(x.median()))
    
    #########################处理一下Column############################
    
    ########Categorical######
    
    Qual_dict = {'Ex':3, 'Gd': 2, 'TA': 1, 'Fa': 0}
    Qual_dict2 = {'Ex':4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po':0}
    Qual_dict3 = {'Ex':4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'No_Basement':0}
    Qual_dict4 = {'Gd':4, 'TA': 3, 'Fa': 2, 'Po': 1, 'No_Basement':0}
    Qual_dict5 = {'Fin':2, 'RFn':1, 'Unf':0}
    
    # GarageCond
    X_prepared['GarageCond'] = X_prepared['GarageCond'].replace(Qual_dict2)
    # GarageQual
    X_prepared['GarageQual'] = X_prepared['GarageQual'].replace(Qual_dict2)
    # GarageFinish
    X_prepared['GarageFinish'] = X_prepared['GarageFinish'].replace(Qual_dict5)
    # HeatingQC
    X_prepared['HeatingQC'] = X_prepared['HeatingQC'].replace(Qual_dict2)
    # BsmtCond
    X_prepared['BsmtCond'] = X_prepared['BsmtCond'].replace(Qual_dict4)
    # BsmtQual
    X_prepared['BsmtQual'] = X_prepared['BsmtQual'].replace(Qual_dict3)
    # ExterCond
    X_prepared['ExterCond'] = X_prepared['ExterCond'].replace(Qual_dict2)
    # ExterQual
    X_prepared['ExterQual'] = X_prepared['ExterQual'].replace(Qual_dict)
    # KitchenQual
    X_prepared['KitchenQual'] = X_prepared['KitchenQual'].replace(Qual_dict)
    # LandSlope
    X_prepared['LandSlope'] = X_prepared['LandSlope'].replace({'Sev':2, 'Mod':1, 'Gtl':0})
    # ONE HOT
    onehot_col = ['MSZoning', 'Neighborhood','Exterior1st','Exterior2nd','Foundation','CentralAir',
                'PavedDrive', 'SaleType','MSSubClass', 'LotShape', 'LandContour','Condition1','BldgType',
                'RoofStyle','Electrical', 'Functional','GarageType'
                    ]
    X_prepared = pd.get_dummies(X_prepared, columns = onehot_col)
    
    ########Numerical########
    
    high_skew_col = ['LotArea', 'WoodDeckSF', '1stFlrSF', 'TotalBsmtSF', 'OpenPorchSF', 'LotFrontage', 
                     'MasVnrArea', 'BsmtFinSF2']
    for col in high_skew_col:
        X_prepared[col] = np.log1p(X_prepared[col])
        
    #########################新feature################################
    
    X['has_2ndsf'] = (X['2ndFlrSF'] != 0).astype(np.int)
    X_prepared['Bath'] = X_prepared['FullBath'] + 0.5 * X_prepared['HalfBath']
    X_prepared['has_pool'] = (X_prepared['PoolArea'] == 0).astype(np.int)
    X_prepared['has_remodel'] = (X_prepared['YearRemodAdd'] != X_prepared['YearBuilt']).astype(np.int)
    X_prepared['BsmtBath'] = X_prepared['BsmtFullBath'] + 0.5 * X_prepared['BsmtHalfBath']
    X_prepared['Overall'] = X_prepared['OverallCond'] + X_prepared['OverallQual']
    X_prepared['has_MasVnr'] = (X_prepared['MasVnrArea'] == 0).astype(np.int)
    X_prepared['Exter'] = X_prepared['ExterCond'] + X_prepared['ExterQual']
    X_prepared['Bsmt'] = X_prepared['BsmtCond'] + X_prepared['BsmtQual']
    X_prepared['has_BsmtFinSF2'] = (X_prepared['BsmtFinSF2'] != 0).astype(np.int)
    X_prepared['has_lowqualitysf'] = (X_prepared['LowQualFinSF'] != 0).astype(np.int)
    X_prepared['Garage'] = X_prepared['GarageQual'] + X_prepared['GarageCond']
    
    #########################不要的column扔了############################
    
    to_drop = ['YearBuilt', 'PoolArea', 'LowQualFinSF']
    X_prepared.drop(to_drop, axis = 1, inplace = True)
    
    #########################test中没有的类型填一下############################
    
    miss_col = [
        'Exterior1st_ImStucc', 'Exterior1st_Stone', 'Exterior2nd_Other'
    ]
    for c in miss_col:
        X_prepared[c] = 0
    return X_prepared

In [294]:
test_prepared = prepare_test(test)
test_prepared = selector.transform(test_prepared)

In [250]:
# for i in test_prepared.columns:
#     if test_prepared[i].isnull().sum() > 0:
#         print(i)

In [296]:
gb_reg.predict(test_prepared)

array([164234.48416268, 192826.44289359, 170815.69712214, ...,
       198110.31218678, 142672.90709447, 218693.07599979])

In [297]:
rf_reg.predict(test_prepared)

array([126746.3675, 161073.1875, 163039.345 , ..., 164785.0925,
       136381.8875, 183409.0825])

In [298]:
xgb_reg.predict(test_prepared)

array([126424.984, 144933.9  , 155037.48 , ..., 172967.34 , 142787.86 ,
       195923.6  ], dtype=float32)

In [299]:
predictions = xgb_reg.predict(test_prepared)
result = pd.DataFrame({'Id': test['Id'], 'SalePrice': predictions})


In [300]:
result.head()


,Id,SalePrice
0,1461,126424.984375
1,1462,144933.906250
2,1463,155037.484375
3,1464,162054.046875
4,1465,227006.156250


In [205]:
# result.to_csv('result.csv',index = False)


In [302]:
# set(X_prepared.columns) - set(test_prepared.columns)